### Creación de un componente básico con Polymer 3.0

In [ ]:
# Se han eliminado las etiquetas 'dom-module' y ahora todo el código se encuentra dentro de una
# clase que hereda de PolymerElement

import {html, PolymerElement} from '@polymer/polymer/polymer-element.js';

 export class BasicElement extends PolymerElement { # siempre es recomendabel exportar nuestros elementos
    
    # por un lado tenemos el template, lo que es una novedad respecto a Polymer 2
  static get template() {
    return html` # vemos que tenemos el tag 'html'
      <style>
        :host {
          display: block;
        }
      </style>
      <h2>Hello [[prop1]]!</h2> # binding normal
    `;
  }
    # aquí tenemos las propiedades
  static get properties() {
    return {
      prop1: {
        type: String,
        value: 'basic-element',
      },
    };
  }
}

window.customElements.define('basic-element', BasicElement); # aquí también cambia la sintaxis respecto a Polymer 2

### Creación de un componente básico con lit-HTML

In [ ]:
# en lugar de importar PolymerElement importamos LitElement
    import {LitElement, html} from '@polymer/lit-element';

    class MyElement extends LitElement {

      static get properties() { 
          return { 
              mood: String 
          }
      }
        
      constructor(){
          super();
          this.mood="hola"
      }

      _render({mood}) {
        return html`<style> .mood { color: green; } </style>
          Web Components are <span class="mood">${mood}</span>!`;
      }

    }

    customElements.define('my-element', MyElement);
  </script>

 

### Custom Elements

#### Diferir trabajos no críticos

Siempre que sea posible, debemos diferir trabajos hasta después de la primera pintura. Para ello podemos usar el módulo 'render-status' que nos provee la utilidad 'afterNextRender' para este propósito

In [ ]:
import {afterNextRender} from '@polymer/polymer/lib/utils/render-status.js';

class DeferElement extends PolymerElement {
  ...
  constructor() {
    super();
    # When possible, use afterNextRender to defer non-critical
    # work until after first paint.
    afterNextRender(this, function() {
      this.addEventListener('click', this._handleClick);
    });
  }
}

#### Extender otros componentes

Importante: si hemos definido un template en el elemento padre del que estamos heredando, podemos heredar su templante en el elemento hijo, si necesidad de que creemos un nuevo template

In [ ]:
'''
OJO!!!! la sintaxis es muy similar a la de Polymer 2 en el caso de extender otros componentes
'''


import {MyElment} from './my-element.js';

export class ExtendedElement extends MyElement { # IMP no olvidar el export en el padre
  static get is() { 
      return 'extended-element'; 
  }

  static get properties() {
    return {
      thingCount: {
        value: 0,
        observer: '_thingCountChanged'
      }
    }
  }
  _thingCountChanged() {
    console.log(`thing count is ${this.thingCount}`);
  }
};

customElements.define(ExtendedElement.is, ExtendedElement);

También podemos extender un template del elemento padre de la siguiente manera:

In [ ]:
'''
OJO!!! para hacer esto tenemos que importar el html:
import {html, PolymerElement} from '@polymer/polymer/polymer-element.js';
''' 
# en el elemento hijo:
static get template() {
  return html`
      <p>This content is from ChildClass.</p>
      <p>${super.template}</p>                  # con esta expresión extendemos el template del padre
      <p>Hello again from ChildClass.</p>`;
}

Otra opción es definir templates parciales en el elemento padre para después heredar en el hijo

In [ ]:
# Elemento padre
export class BaseClass extends PolymerElement {
  static get template() {
    return html`
      <div>${this.headerTemplate}</div> # template parcial
      <p>Hello this is some content</p>
      <div>${this.footerTemplate}</div> # template parcial
    `;
  }
  static get headerTemplate() { return html`<h1>BaseClass: Header</h1>` } # template parcial
  static get footerTemplate() { return html`<h1>BaseClass: Footer</h1>` } # template parcial
}

# Elemento hijo
export class ChildClass extends BaseClass {
    static get headerTemplate() { return html`<h2>ChildClass: Header</h2>` }
    static get footerTemplate() { return html`<h2>ChildClass: Footer</h2>` }
  }

#### Mixins

In [ ]:
# crear un mixin con la función flecha. OJO!!! no olvidar el export const / let!!!!:
export const MyMixin = (superClass) => class extends superClass {
  ...
}

# si deseamos hacer un paquete con un mixin para después distribuirlo se recomienda lo siguiente:
# - usar la función dedupingMixin para asegurarnos que el mixin sólo se aplicará una vez
# - definir el mixin en un módulo y exportarlo
import {dedupingMixin} from '@polymer/polymer/lib/utils/mixin.js';

# definimos el mixin. OJO!!! no olvidar el let!!!!
let internalMixinB = (base) =>
  class extends base {
    ...
  }

# lo deduplicamos y exportamos 


export const MixinB = dedupingMixin(internalMixinB);

# lo usamos dentro de otro componente
import {mixinB} from './mixin-b.js';

class Foo extends MixinB(PolymerElement) { ... }

#### Valores por defecto de una propiedad

Los valores por defecto de una propiedad los podemos definir o bien dentro del objeto 'properties' en el campo 'value' o de manera imperativa en el 'constructor':

In [ ]:
# en el objeto 'properties'
static get properties() {
    return {
      mode: {
        type: String,
        value: 'auto'
      },
    }
}

# en el constructor
constructor() {
  super();  
  this.mode = 'auto';
  this.data = {};
}

#### Notificación de los cambios del valor de una propiedad a través de eventos

Si queremos que se lance un evento cada vez que cambie el valor de una propiedad, el 'notify' tiene que estar definido como 'true'. En ese caso, se lanzará el evento 'nombre-propiedad-changed'.
Importante, este evento no hace burbujas, por lo que el manejador de eventos debe ser agregado directamente al elemento que genera el evento

### Shadow DOM & Estilos

#### Uso del 'slot'

Por defecto, si un elemento tiene shadow DOM el shadow tree es renderizado en vez del hijo del elemento:

In [ ]:
# imaginemos que tenemos un elemento <my-header> con el siguiente shadow tree:
<header>
  <h1><slot></slot></h1>
  <button>Menu</button>
</header>

# y ahora imaginemos que a ese elemente, que tiene el shadow tree anterior, le agregamos un hijo texto
<my-header>Shadow DOM</my-header>

# como hemos puesto el <slot> entre las <h1>, el hijo anterior 'Shadow DOM' se renderizará 
# en el espacio del slot, así (el nuevo árbol se llamará 'light DOM':
<my-header>
  <header>
    <h1>Shadow DOM</h1> ## aquí es donde estaba el slot, y ahora está 'Shadow DOM'
    <button>Menu</button>
  </header>
</my-header>

In [ ]:
# podemos determinar que un slot se muestre sólo si lo definimos explícitamente con el atributo 'name'
<slot-element>
      <span slot="title"> este es el slot con el name="title"</span>
</slot-element>

# código de nuestro 'slot-element'
<p><slot name="title"></slot></p>

# vemos que el mensaje del span cuyo slot = 'title' se mostrará en el slot con name='title'

#### Estilos

Aunque el 'shadow tree' encapsula los estilos de un elemento del exterior, existen algunas propiedades de estilo hereditarias que aún se heredan desde el elemento 'host' hasta el 'shadow tree':

In [ ]:
# en nuestro index.html, donde tenemos nuestro mi-elemento.js, definimos el siguiente estilo para el body
<style>
    body{color:red;}
</style>

# en mi-elemento.js tenemos otros estilos, pero no hemos definido el estilo 'color', por lo que se
# heredará el color:red del elemento host, que en este caso el index.html

Hay otra manera en la que el 'shadow tree' de un elemento es alcanzado por estilos de otro elemento, en este caso, de su elemento 'host'

In [ ]:
# imaginemos que tenemos un elemento 'host-element' y dentro tenemos el elemento 'styles-element'
# dentro de styles-element hemos definido el siguiente estilo en el selector :host
:host(.warning){
 	background-color:yellow
}
# si en el host-element insertamos nuestro styles-element con la clase .warning:
<styles-element class="warning"></styles-element>

#### Theming y Custom Properties

Podemos definir una API de estilos para nuestro componente a través de las custom properties, que son una especie de variables CSS:

In [ ]:
# tenemos nuestro styles-element donde definimos una custom property con un valor por defecto
:host{
    background-color:var(--mi-color, blue)
}

# desde su componente host le podemos dar cualquier valor a la variable --mi-color
:host {
    --mi-color:red;
}

#### Mixins de Custom Properties

Básicamente un mixin es una variable que contiene múltiples propiedades:

In [ ]:
'''
IMPORTANTÍSIMO: para usar los mixins debemos importar el polyfill
import '@webcomponents/shadycss/entrypoints/apply-shim.js';
'''

# en el host-element definimos el mixin:
<style>
    :host {
        --mi-mixin:{
          	color:blue
          }   
   	}

</style>

# aplicamos el mixin en el hijo:
:host {
    @apply --mi-mixin
}


#### Estilo para el slot

In [ ]:
# es necesario que definamos en el componente:
static get template() {
  return html`
    <style>
      p ::slotted(*), h1 ::slotted(*) { # el selector ::slotted
        font-family: sans-serif;
        color:green;
      }
    </style>
    <h1><div><slot name='heading1'></slot></div></h1>
    <p><slot name='para'></slot></p>
  `;
}


# en el padre
<head>
  <script type="module" src="custom-element.js"></script>
</head>
<body>  
  <custom-element>
      <div slot="heading1">Heading 1. I'm green.</div>
      <div slot="para">Paragraph text. I'm green too.</div>
    </custom-element>
</body>

#### Compartir estilos entre elementos

En actualización

#### Cambiar estilos dinámicamente

podemos modificar dinámicamente el ***custom style*** de un componente llamando al método updateStyles()

In [ ]:
'''
IMPORTANTÍSIMO
import '@webcomponents/shadycss/entrypoints/apply-shim.js';
'''

<style>
    :host {
        color:var(--mi-color, blue) # color azul por defecto en el custom style
    }
</style>

<button on-click="changeTheme">Change theme</button> # color azul. Cuando clickemos el botón cambia el color

############

changeTheme(){
  this.updateStyles({
      '--mi-color':'red' # cambiamos el custom style a rojo
  })
}

### Sistema de datos

Polymer nos permite observar cambios en las propiedades de un elmento y realizar varias acciones basadas en los cambios de los datos. Estas acciones, o efectos de una propiedad, incluyen:

* Observers, que son callbacks invocados cuando un dato cambia.

* Computed properties, que son propiedades virtuales computadas, basadas en otras propiedades, y recomputadas cuando los input data cambian.

* Binding, que son anotaciones que actualizan las propiedades, atributos, o texto de un nodo del DOM cuando un dato cambia

Cada elemento de Polymer maneja si propio modelo de datos y los elementos de su DOM local. El modelo de este elemento son sus propiedades. El binding permite linkar el modelo de un elemento con los elementos en su DOM local.

El sistema de datos está basado en ***paths***, que no objetos, donde el path representa una propiedad o subpropiedad relativa al elemento host. 



In [ ]:
class NameCard extends PolymerElement {
  constructor() {
    super();
    this.name = {first: 'Kai', last: 'Li'};
  }
  static get template() {
    return html `
      <div>[[name.first]] [[name.last]]</div> # paths
    `;
  }
}
customElements.define('name-card', NameCard);

#### Cambios observables y no observables

Ojo, hay cambios que Polymer no puede detectar y, por lo tanto, no puede asociarlos a un path. Otros, los llamados ***cambios observables*** son detectados automáticamente:

* Settear una propiedad directamente a un elemento

```
this.owner = 'Jane'; 
```

un observer, el binding o una computed property detectarán este cambio

* Settear la subpropieddad de un elemento con un binding de 2 direcciones

```
<local-dom-child name="{{hostProperty.subProperty}}"></local-dom-child>
```

Entre los cambios ***no observables estarían los siguientes***:

* Mutar imperativamente la subpropiedad de un objeto:

```
this.address.street = 'Elm Street';
```
lo deberíamos realizar usando métodos de Polymer:
```
this.set('address.street', 'Half Moon Street');
```

* Mutar imperativamente un array:

```
this.users.push({ name: 'Maturin'});
```
lo deberíamos hacer así:
```
this.push('users', { name: 'Maturin'});
```

Polymer nos ofrece los siguientes métodos para que las mutaciones en un Array sean observables:

```
push(path, item1, [..., itemN])
pop(path)
unshift(path, item1, [..., itemN])
shift(path)
splice(path, index, removeCount, [item1, ..., itemN])
```


Hay situaciones en la que los métodos anteriores no son válidos, por ejemplo cuando usamos una librería de terceros. En este caso, para notificar los cambios en un Array lo haríamos de la siguiente manera:

```
this.notifyPath('address.street');
```
usaremos notifyPath() cuando necesitemos usar el path exacto que cambió, en el resto de casos usaremos notifySplices()



#### Setear varias propiedades a la vez

In [ ]:
## el observer se dispara 2 veces
this.a = 10;
this.b = 20;

## el observer se dispara 1 vez
this.setProperties({a: 10, b: 20});

#### Observers

In [ ]:
# Observer simple:

class XCustom extends PolymerElement {
  static get properties() {
    return {
      active: {
        type: Boolean,
        // Observer method identified by name
        observer: '_activeChanged'
      }
    }
  }
  // Observer method defined as a class method
  _activeChanged(newValue, oldValue) {
    this.toggleClass('highlight', newValue);
  }
}

# Observer complejo:

static get observers() {
  return [
    # Observer method name, followed by a list of dependencies, in parenthesis
    'userListChanged(users.*, filter)'
  ]
}

# Observer complejo, observar varias propiedades a la vez:

class XCustom extends PolymerElement {

  static get properties() {
    return {
        preload: Boolean,
        src: String,
        size: String
    }
  }

  # Each item of observers array is a method name followed by
  # a comma-separated list of one or more dependencies.
  static get observers() {
    return [
        'updateImage(preload, src, size)'
    ]
  }

  # Each method referenced in observers must be defined in
  # element prototype. The arguments to the method are new value
  # of each dependency, and may be undefined.
  updateImage(preload, src, size) {
    # ... do work using dependent values
  }
}

# Observer complejo, observar una subropiedad de un objeto

  static get observers() {
    return [
        'userNameChanged(user.name)'
    ]
  }

# Observer complejo, observar la mutación de un Array

static get observers() {
  return [
    'usersAddedOrRemoved(users.splices)' #(path.splices)
  ]
}

#### Propiedades computadas

Las propiedades computadas son aquellas que toman su valor por medio de un cómputo, al que se llega mediante los valores de otras propiedades del componente.

In [ ]:
static get properties() {
  return {
    nombre: String,
    apellidos: String,
    nombreCompleto: {
      type: String,
      computed: 'definirNombreCompleto(nombre, apellidos)'
    }
  };
}

definirNombreCompleto(nombre, apellidos) {
  return nombre + ' ' + apellidos;
}

#### Binding computado

Es similar a una propiedad computada

In [ ]:
class XCustom extends PolymerElement {
  static get properties() {
    return {
      given: String,
      family: String
    };
  }
  
  _formatName(given, family) { # definimos la función que devolverá el binding computado
    return `${family}, ${given}`;  # Ojo!! aqui ya definimos el literal template!!
  }
  
  static get template() {
    return html`
      My name is <span>[[_formatName(given, family)]]</span>
    `;
  }
}

### Helper Elements

#### Dom-repeat

Ojo, debemos importar el template repeater mixin:
```
import '@polymer/polymer/lib/elements/dom-repeat.js';
```


Cómo manejar ***eventos*** dentro de un dom-repeat:
Cuando agreguemos un manejador de eventos de manera declarativa dentro de un dom-repeat, el repeater agrega la propiedad ***model** para cada evento enviado al manejador de eventos, el cual contiene el scope donde se ha generado la instancia del template, por lo que el item del dato será el ***model.item***

In [ ]:
class XCustom extends PolymerElement {
  static get properties() {
    return {
      menuItems: {
        type: Array,
        value() {
          return [
            {name: 'Pizza', ordered: 0},
            {name: 'Pasta', ordered: 0},
            {name: 'Toast', ordered: 0}
          ];
        }
      }
    };
  }
  order(e) {
    e.model.set('item.ordered', e.model.item.ordered+1);
  }                                            
  static get template() {
    return html`
      <template is="dom-repeat" id="menu" items="{{menuItems}}">
        <div>
          <span>{{item.name}}</span>
          <span>{{item.ordered}}</span>
          <button on-click="order">Order</button>
        </div>
      </template>
    `;
  }
}

#### Filtrado y ordenado de listas

Podemos filtrar u ordenar una lista devuelta por un dom-repeat, especificando las propiedades ***filter*** y ***sort***

In [ ]:
# por ejemplo, tenemos el siguiente filtro, que actua como un observer:
isEngineer(item) {
  return item.type == 'engineer' || item.manager.type == 'engineer';
}

# en el dom-repeat incluimos el fitro dentro del atributo 'filter'
<template is="dom-repeat" items="{{employees}}"
    filter="isEngineer" observe="type manager.type">
    


#### Filtrado y ordenado dinámico de listas

En ocasiones, queremos que esa propiedad filtro u ordenar sean dinámicas, para lo que usaremos un computed binding:

In [ ]:
class XCustom extends PolymerElement {
  static get properties() {
    return {
      employees: {
        type: Array,
        value() {
          return [
            { firstname: "Jack", lastname: "Aubrey" },
            { firstname: "Anne", lastname: "Elliot" },
            { firstname: "Stephen", lastname: "Maturin" },
            { firstname: "Emma", lastname: "Woodhouse" }
          ];
        }
      }
    };
  }
  order(e) {
    e.model.set('item.ordered', e.model.item.ordered+1);
  }
  computeFilter(string) {
    if (!string) {
      # setea el filtro a nulo para deshabilitarlo 
    } else {
      # devuelve una función filtro para el string de búsqueda actual 
      string = string.toLowerCase();
      return function(employee) {
        var first = employee.firstname.toLowerCase();
        var last = employee.lastname.toLowerCase();
        return (first.indexOf(string) != -1 ||
            last.indexOf(string) != -1);
      };
    }
  }
  static get template() {
    return html`
      Search string: <input value="{{searchString::input}}"><br/><br/>
      <!-- 
        computeFilter returns a new filter function 
        whenever searchString changes 
      -->
      <template is="dom-repeat" items="{{employees}}" as="employee"
        filter="{{computeFilter(searchString)}}"> # filtro como binding computado
        <div>{{employee.lastname}}, {{employee.firstname}}</div>
      </template>
    `;
  }
}

#### Dom-repeat dentro de Dom-repeat (ver ejemplo vigio-prices)

In [ ]:
static get template() {
  return html`
    <template is="dom-repeat" items="{{employees}}" as="employee">
      <b>Employee {{index}}</b>
      <div>First name: <span>{{employee.firstname}}</span></div>
      <div>Last name: <span>{{employee.lastname}}</span></div>
      <div>Direct reports:</div>
      <template is="dom-repeat" items="{{employee.reports}}" as="report" index-as="report_no">
        <div>
        <span>{{report_no}}</span>.
        <span>{{report.firstname}}</span> <span>{{report.lastname}}</span>
        </div>
      </template>
      <br />
    </template>
  `;
}

# si tenemos problemas de rendimiento podemos incluir la propiedad initialCount en el dom-repeat

#### Seleccionar un item dentro de un Array (dom-repeat)

In [ ]:
# usaremos el elemento array-selector
# OJO!!!! tenemos que importar su mixin import '@polymer/polymer/lib/elements/array-selector.js';

class XCustom extends PolymerElement {
  static get properties() {
    return {
      employees: {
        type: Array,
        value() {
          return [
            {given: 'Kamil', family: 'Smith'},
            {given: 'Sally', family: 'Johnson'},
            {given: 'Shauna', family: 'Bell'},
            {given: 'San', family: 'Zhang'},
            {given: 'Carlo', family: 'Lopez'}
          ];
        }
      }
    };
  }
  toggleSelection(e) {
    var item = this.$.employeeList.itemForElement(e.target);
    this.$.selector.select(item);
  }
  static get template() {
    return html`
      <div><b>All employees</b></div><br />
      <template is="dom-repeat" id="employeeList" items="{{employees}}">
          <div>Given name: <span>{{item.given}}</span></div>
          <div>Family name: <span>{{item.family}}</span></div>
          <button on-click="toggleSelection">Select/deselect</button>
          <br /><br />
      </template>
      <array-selector id="selector" items="{{employees}}" selected="{{selected}}" multi toggle></array-selector>
      <div><b>Selected employees</b></div><br />
      <template is="dom-repeat" items="{{selected}}">
          <div>Given name: <span>{{item.given}}</span></div>
          <div>Family name: <span>{{item.family}}</span></div>
          <br />
      </template>
    `;
  }
}
customElements.define('x-custom', XCustom);

#### Dom-if

Ojo! tenemos que importar su mixin:
```
import '@polymer/polymer/lib/elements/dom-if.js';
```
Si queremos setear imperativamente la propiedad if:
```
var conditional = document.querySelector('dom-if');
conditional.if = true;
````

